In [1]:
import pandas as pd

In [70]:
data = pd.read_csv('input',header=None)
print(data.head())
orders = [ (d[0][0],int(d[0][1:])) for d in data.values.tolist()]
orders[:10]

     0
0  L90
1  F82
2  R90
3  F57
4  L90


[('L', 90),
 ('F', 82),
 ('R', 90),
 ('F', 57),
 ('L', 90),
 ('F', 21),
 ('W', 4),
 ('S', 2),
 ('L', 90),
 ('W', 5)]

In [71]:
# DAY 12 - PART 1

import numpy as np
from functools import reduce, partial

start_position = (np.array([0,0]),'E')

rotate = 'ESWN'

vector_of = {
    'N': np.array([0,1]),
    'S': np.array([0,-1]),
    'E': np.array([1,0]),
    'W': np.array([-1,0])
}

def go(position, direction, command, distance):
    return (position + (vector_of[direction if command == 'F' else command]*distance), direction)

def turn(position, direction, command, degree, rotation_direction=1):
    return (position, rotate[(rotate.index(direction) + (int(degree/90) % 4)*rotation_direction) % 4])

order_translation = {
    'F': go,
    'R': turn,
    'L': partial(turn, rotation_direction=-1),
    'N': go,
    'E': go,
    'W': go,
    'S': go
}

end_position, end_direction = reduce((lambda p,o: order_translation[o[0]](*p,*o)), orders, start_position)

sum([abs(v) for v in end_position])

962

In [72]:
# DAY 12 - PART 2
import numpy as np
from functools import reduce, partial

# waypoint position is distinguished by units in N,E direction
# i.e. (10,1) => 10 East, 1 North
#      (-5,3) => 5 West, 3 North
#      (2,-6) => 2 East, 6 South
start_position = (np.array([0,0]), np.array([10,1]))

move = {
    'N': np.array([0,1]),
    'S': np.array([0,-1]),
    'E': np.array([1,0]),
    'W': np.array([-1,0])
}
                  
rotate = {
    # rotating to the left works like this: (x,y) => (-y, x)
    'L': (lambda waypoint: (waypoint*np.array([1,-1]))[::-1]),
    # rotating to the right works like this: (x,y) => (y,-x)
    'R': (lambda waypoint: (waypoint*np.array([-1,1]))[::-1])
}

def forward(boat, waypoint, command, distance):
    print(f'{command}{distance} :: Moving boat {boat} => {boat + (waypoint*distance)}')
    return (boat + (waypoint*distance), waypoint)

def move_waypoint(boat, waypoint, command, distance):
    print(f'{command}{distance} :: Moving waypoint {waypoint} => {waypoint+(move[command]*distance)}')
    return (boat, waypoint+(move[command]*distance))

def turn_waypoint(boat, waypoint, command, degree):
    times = int(degree/90) % 4 # number of times we have to apply rotation
    # maybe there is a not so funky way of applying the same function n times to a value than that ¯\_(ツ)_/¯
    new_waypoint = reduce((lambda wp,rotation: rotation(wp)), [rotate[command] for i in range(times)], waypoint)
    print(f'{command}{degree} :: turning {times} times {command} wp: {waypoint} => {new_waypoint}')
    return (boat, new_waypoint)
                  
order_translation = {
    'F': forward,
    'R': turn_waypoint,
    'L': turn_waypoint,
    'N': move_waypoint,
    'E': move_waypoint,
    'W': move_waypoint,
    'S': move_waypoint
}

end_position, end_direction = reduce((lambda p,o: order_translation[o[0]](*p,*o)), orders, start_position)

sum([abs(v) for v in end_position])

L90 :: turning 1 times L wp: [10  1] => [-1 10]
F82 :: Moving boat [0 0] => [-82 820]
R90 :: turning 1 times R wp: [-1 10] => [10  1]
F57 :: Moving boat [-82 820] => [488 877]
L90 :: turning 1 times L wp: [10  1] => [-1 10]
F21 :: Moving boat [488 877] => [ 467 1087]
W4 :: Moving waypoint [-1 10] => [-5 10]
S2 :: Moving waypoint [-5 10] => [-5  8]
L90 :: turning 1 times L wp: [-5  8] => [-8 -5]
W5 :: Moving waypoint [-8 -5] => [-13  -5]
R90 :: turning 1 times R wp: [-13  -5] => [-5 13]
F84 :: Moving boat [ 467 1087] => [  47 2179]
E1 :: Moving waypoint [-5 13] => [-4 13]
R90 :: turning 1 times R wp: [-4 13] => [13  4]
S4 :: Moving waypoint [13  4] => [13  0]
F13 :: Moving boat [  47 2179] => [ 216 2179]
E3 :: Moving waypoint [13  0] => [16  0]
N2 :: Moving waypoint [16  0] => [16  2]
W4 :: Moving waypoint [16  2] => [12  2]
F33 :: Moving boat [ 216 2179] => [ 612 2245]
W1 :: Moving waypoint [12  2] => [11  2]
F48 :: Moving boat [ 612 2245] => [1140 2341]
E4 :: Moving waypoint [11  2] =

E4 :: Moving waypoint [83  5] => [87  5]
L180 :: turning 2 times L wp: [87  5] => [-87  -5]
N2 :: Moving waypoint [-87  -5] => [-87  -3]
L90 :: turning 1 times L wp: [-87  -3] => [  3 -87]
N1 :: Moving waypoint [  3 -87] => [  3 -86]
E2 :: Moving waypoint [  3 -86] => [  5 -86]
S3 :: Moving waypoint [  5 -86] => [  5 -89]
F63 :: Moving boat [45096 21413] => [45411 15806]
W1 :: Moving waypoint [  5 -89] => [  4 -89]
F90 :: Moving boat [45411 15806] => [45771  7796]
S3 :: Moving waypoint [  4 -89] => [  4 -92]
L270 :: turning 3 times L wp: [  4 -92] => [-92  -4]
E4 :: Moving waypoint [-92  -4] => [-88  -4]
F19 :: Moving boat [45771  7796] => [44099  7720]
R90 :: turning 1 times R wp: [-88  -4] => [-4 88]
E5 :: Moving waypoint [-4 88] => [ 1 88]
L180 :: turning 2 times L wp: [ 1 88] => [ -1 -88]
S1 :: Moving waypoint [ -1 -88] => [ -1 -89]
R90 :: turning 1 times R wp: [ -1 -89] => [-89   1]
F45 :: Moving boat [44099  7720] => [40094  7765]
R90 :: turning 1 times R wp: [-89   1] => [ 1 89]

56135